<a href="https://colab.research.google.com/github/c-brun/traffic-sign-detection/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List
from enum import Enum, auto
import random

class GameConstants:
    GRID_WIDTH = 4
    GRID_HEIGHT = 4
    GOLD_REWARD = 1000
    DEATH_PENALTY = -1000
    ACTION_COST = -1
    ARROW_COST = -10
    START_X = 0
    START_Y = 0

In [ ]:
class Percept():
    time_step: int
    stench: bool
    breeze: bool
    glitter: bool
    bump: bool
    scream: bool
    done: bool
    reward: int

    def __init__(self, time_step: int, stench: bool, breeze: bool, glitter: bool, bump: bool, scream: bool, done: bool, reward: int):
        self.time_step = time_step
        self.stench = stench
        self.breeze = breeze
        self.glitter = glitter
        self.bump = bump
        self.scream = scream
        self.done = done
        self.reward = reward

    def __str__(self):
        # add helper function to return the contents of a percept in a readable form
        return f'Time step: {self.time_step}\nBump: {self.bump}\nBreeze: {self.breeze}\nStench: {self.stench}\nScream: {self.scream}\nGlitter: {self.glitter}\nReward: {self.reward}\nDone: {self.done}'

In [ ]:
class Action(Enum):
    LEFT = 0
    RIGHT = 1
    FORWARD = 2
    GRAB = 3
    SHOOT = 4
    CLIMB = 5

In [ ]:
class Orientation(Enum):
    E = 0
    S = 1
    W = 2
    N = 3

    def symbol(self) -> str:
        return self.name

        # code for function to return the letter code ("E", "S", etc.) of this instance of an orientation
        # You could create a __str__(self) for this instead of the symbol function if you prefer

    def turn_right(self) -> 'Orientation':
        match self.value:
            case 0: return Orientation.S
            case 1: return Orientation.W
            case 2: return Orientation.N
            case 3: return Orientation.E

        # Note: the quotes around the type Orientation are because of a quirk in Python.  You can't refer
        # to Orientation without quotes until it is defined (and we are in the middle of defining it)

    def turn_left(self) -> 'Orientation':
        match self.value:
            case 0: return Orientation.N
            case 1: return Orientation.E
            case 2: return Orientation.S
            case 3: return Orientation.W

In [ ]:
class NaiveAgent:

    def choose_action(self):
        match random.randint(0, 5):
            case 0: return Action.LEFT
            case 1: return Action.RIGHT
            case 2: return Action.FORWARD
            case 3: return Action.GRAB
            case 4: return Action.SHOOT
            case 5: return Action.CLIMB
        # return a random action

    def run(self):
      """Run the agent in the given environment"""
      # Display the legend once at the start
      print("=" * 50)
      print("WUMPUS WORLD SIMULATION")
      print("=" * 50)
      print("LEGEND:")
      print("  A→ A← A↑ A↓  = Agent (facing direction)")
      print("  P             = Pit")
      print("  W             = Wumpus (alive)")
      print("  †W            = Wumpus (dead)")
      print("  G             = Gold")
      print("  Multiple objects in same cell shown as: A→,G")
      print("=" * 50)

      # Get initial percept and start the game
      percept = env.get_initial_percept()
      env.visualize(percept)

      while not percept.done:
          action = self.choose_action()
          percept = env.step(action)
          env.visualize(percept)

In [ ]:
class Location:
    x: int
    y: int

    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def __str__(self):
        return f'({self.x}, {self.y})'


    def is_left_of(self, location: 'Location')->bool:
        return self.x == location.x - 1
    # return True if self is just left of given location

    def is_right_of(self, location: 'Location')->bool:
        return self.x == location.x + 1
    # return True if self is just right of given location

    def is_above(self, location: 'Location')->bool:
        return self.y == location.y + 1
    # return True if self is immediately above given location

    def is_below(self, location: 'Location')->bool:
        return self.y == location.y - 1
    # return True if self is immediately below given location

    def neighbours(self)->List['Location']:
        neighbours = []
        if self.x > 0:
            neighbours.append(Location(self.x - 1, self.y))
        if self.x < GameConstants.GRID_WIDTH - 1 :
            neighbours.append(Location(self.x + 1, self.y))
        if self.y > 0:
            neighbours.append(Location(self.x, self.y - 1))
        if self.y < GameConstants.GRID_HEIGHT - 1:
            neighbours.append(Location(self.x, self.y + 1))
        return neighbours
    # return list of neighbour locations

    # return True if location given is self's location
    def is_location(self, location: 'Location')->bool:
        return self.x == location.x and self.y == location.y

    def at_left_edge(self) -> bool:
        return self.x == 0
    # return True if at the left edge of the grid


    def at_right_edge(self) -> bool:
        return self.x == GameConstants.GRID_WIDTH - 1
    # return True if at the right edge of the grid


    def at_top_edge(self) -> bool:
        return self.y == GameConstants.GRID_HEIGHT - 1
    # return True if at the top edge of the grid


    def at_bottom_edge(self) -> bool:
        return self.y == 0
     # return True if at the bottom edge of the grid


    def forward(self, orientation) -> bool:
        old_x, old_y = self.x, self.y  # Remember where you were

        match orientation:
            case Orientation.E: self.x = min(GameConstants.GRID_WIDTH - 1, self.x + 1)
            case Orientation.S: self.y = max(0, self.y - 1)
            case Orientation.W: self.x = max(0, self.x - 1)
            case Orientation.N: self.y = min(GameConstants.GRID_HEIGHT - 1, self.y + 1)

        return (old_x == self.x and old_y == self.y)  # Return True if bumped a wall
    # modify self.x and self.y to reflect a forward move and return True if bumped a wall


    def set_to(self, location: 'Location'):
        self.x = location.x
        self.y = location.y
    # set self.x and self.y to the given location


    @staticmethod
    def from_linear(n: int) -> 'Location':
        return Location(n % GameConstants.GRID_WIDTH, n // GameConstants.GRID_WIDTH)
    # convert an index from 0 to 15 to a location


    def to_linear(self)->int:
        return self.y * GameConstants.GRID_WIDTH + self.x
    # convert self to an index from 0 to 15


    @staticmethod
    def random() -> 'Location':
        valid_locations = []
        for x in range(GameConstants.GRID_WIDTH):
            for y in range(GameConstants.GRID_HEIGHT):
                if not (x == 0 and y == 0):  # Exclude starting position
                    valid_locations.append(Location(x, y))

        return random.choice(valid_locations)

In [ ]:
class Environment:
    wumpus_location: Location
    wumpus_alive: bool
    agent_location: Location
    agent_orientation: Orientation
    agent_has_arrow: bool
    agent_has_gold: bool
    game_over: bool
    gold_location: Location
    pit_locations: List[Location]
    time_step: int
    cumulative_reward: int

    def __init__(self, pit_prob: float, allow_climb_without_gold: bool):
        self.make_wumpus()
        self.make_gold()
        self.make_pits(pit_prob)
        self.agent_location = Location(GameConstants.START_X, GameConstants.START_Y)
        self.agent_orientation = Orientation.E
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.game_over = False
        self.time_step = 0
        self.cumulative_reward = 0
        self.last_action = None
        self.last_reward = 0
        self.allow_climb_without_gold = allow_climb_without_gold

        # initialize the environment state variables (use make functions below)

    def get_initial_percept(self) -> Percept:
        return Percept(
            self.time_step,           # Should be 0 initially
            self.is_stench(),
            self.is_breeze(),
            self.is_glitter(),
            False,                    # No bump initially
            False,                    # No scream initially
            self.game_over,
            0                         # No action penalty for initial observation
        )

    def make_wumpus(self):
        self.wumpus_location = Location.random() # eliminates the bottom left corner
        self.wumpus_alive = True
        # choose a random location for the wumpus (not bottom left corner) and set it to alive

    def make_gold(self):
          self.gold_location = Location.random() # eliminates the bottom left corner
        # choose a random location for the gold (not bottom left corner)

    def make_pits(self, pit_prob: float):
        n = GameConstants.GRID_WIDTH * GameConstants.GRID_HEIGHT
        self.pit_locations = []
        temp = [num for num in range(1, n) if random.random() < pit_prob]
        for i in range(len(temp)):
            self.pit_locations.append(Location.from_linear(temp[i]))
        # create pits with prob pit_prob for all locations except the bottom left corner

    def is_pit_at(self, location: Location) -> bool:
        for pit in self.pit_locations:
            if pit.is_location(location):
                return True
        return False
        # return true if there is a pit at location

    def is_pit_adjacent_to_agent(self) -> bool:
        """True if pit exists in adjacent cells"""
        for adj in self.agent_location.neighbours():
            if self.is_pit_at(adj):
                return True
        return False
        # return true if there is a pit above, below, left or right of agent's current location

    def is_wumpus_adjacent_to_agent(self) -> bool:
        for wumpus in self.wumpus_location.neighbours():
            if wumpus.is_location(self.agent_location):
                return True
        return False
        # return true if there is a wumpus adjacent to the agent

    def is_agent_at_hazard(self)->bool:
        if self.wumpus_location.is_location(self.agent_location):
                return True
        for pit in self.pit_locations:
            if pit.is_location(self.agent_location):
                return True
        return False
        # return true if the agent is at the location of a pit or the wumpus

    def is_wumpus_at(self, location: Location) -> bool:
        return self.wumpus_location.is_location(location)
        # return true if there is a wumpus at the given location

    def is_agent_at(self, location: Location) -> bool:
        return self.agent_location.is_location(location)
        # return true if the agent is at the given location

    def is_gold_at(self, location: Location) -> bool:

        if self.agent_has_gold:
            return True

        if self.gold_location is None:
            return False  # No gold exists in the world anymore
        return self.gold_location.is_location(location)

    def is_glitter(self) -> bool:
        if self.gold_location is None:
            return False  # No gold to glitter
        return self.gold_location.is_location(self.agent_location)
        # return true if the agent is where the gold is

    def is_breeze(self) -> bool:
        return self.is_pit_adjacent_to_agent()
        # return true if one or more pits are adjacent to the agent.

    def is_stench(self) -> bool:
        if self.is_wumpus_adjacent_to_agent() or self.is_wumpus_at(self.agent_location):
            return True
        return False
        # return true if the wumpus is adjacent to the agent or the agent is in the room with the wumpus

    def wumpus_in_line_of_fire(self) -> bool:
        match self.agent_orientation.value:
            case Orientation.W.value: return self.agent_location.x > self.wumpus_location.x and self.agent_location.y == self.wumpus_location.y
            case Orientation.E.value: return self.agent_location.x < self.wumpus_location.x and self.agent_location.y == self.wumpus_location.y
            case Orientation.S.value: return self.agent_location.x == self.wumpus_location.x and self.agent_location.y > self.wumpus_location.y
            case Orientation.N.value: return self.agent_location.x == self.wumpus_location.x and self.agent_location.y < self.wumpus_location.y
        # return true if the wumpus is a cell the arrow would pass through if fired

    def kill_attempt(self) -> bool:
        if self.agent_has_arrow:
            self.agent_has_arrow = False
            if self.wumpus_in_line_of_fire() and self.wumpus_alive:
                self.wumpus_alive = False
                return True
        return False

        # return true if the wumpus is alive and in the line of fire
        # if so set the wumpus to dead

    def create_standard_percept(self, reward: int, bump: bool = False, scream: bool = False) -> Percept:
        self.cumulative_reward += reward
        self.last_reward = reward

        return Percept(self.time_step, self.is_stench(), self.is_breeze(), self.is_glitter(), bump, scream, self.game_over, reward)

    def handle_turn_left(self) -> Percept:
        self.agent_orientation = self.agent_orientation.turn_left()
        return self.create_standard_percept(GameConstants.ACTION_COST)

    def handle_turn_right(self) -> Percept:
        self.agent_orientation = self.agent_orientation.turn_right()
        return self.create_standard_percept(GameConstants.ACTION_COST)

    def handle_grab(self) -> Percept:
        if self.is_gold_at(self.agent_location):
            self.agent_has_gold = True
            # Remove gold from world - it's now in agent's inventory, not on the ground
            self.gold_location = None

        return self.create_standard_percept(GameConstants.ACTION_COST)

    def handle_climb(self) -> Percept:
        if self.agent_location.is_location(Location(GameConstants.START_X, GameConstants.START_Y)):
            if self.agent_has_gold:
                self.game_over = True
                return self.create_standard_percept(GameConstants.GOLD_REWARD + GameConstants.ACTION_COST)
            elif self.allow_climb_without_gold:
                self.game_over = True
                return self.create_standard_percept(GameConstants.ACTION_COST)
        return self.create_standard_percept(GameConstants.ACTION_COST)

    def handle_forward(self) -> Percept:
        bump = self.agent_location.forward(self.agent_orientation)
        if self.is_pit_at(self.agent_location) or (self.is_wumpus_at(self.agent_location) and self.wumpus_alive):
            self.game_over = True
            return self.create_standard_percept(GameConstants.DEATH_PENALTY + GameConstants.ACTION_COST)
        else:
            return self.create_standard_percept(GameConstants.ACTION_COST)

    def handle_shoot(self) -> Percept:
        if self.agent_has_arrow:
            scream = self.kill_attempt()
            return self.create_standard_percept(GameConstants.ARROW_COST + GameConstants.ACTION_COST)
        else:
            return self.create_standard_percept(GameConstants.ACTION_COST)

    def step(self, action: Action) -> Percept:
        self.time_step += 1
        self.last_action = action  # Store the action

        match action:
            case Action.LEFT: return self.handle_turn_left()
            case Action.RIGHT: return self.handle_turn_right()
            case Action.GRAB: return self.handle_grab()
            case Action.CLIMB: return self.handle_climb()
            case Action.SHOOT: return self.handle_shoot()
            case Action.FORWARD: return self.handle_forward()
            # for each of the actions, make any agent state changes that result and return a percept including the reward

    def get_cumulative_reward(self) -> int:
        return self.cumulative_reward

    def visualize(self, current_percept=None):
        print("\n" + "="*80)
        title = f"WUMPUS WORLD - Step {self.time_step}"
        if hasattr(self, 'last_action') and self.last_action is not None:
            title += f" - {self.last_action}"
        print(title)
        print("="*80)

        # Print grid
        print("┌───────┬───────┬───────┬───────┐")

        # Outer loop: iterate through rows (y-coordinates from top to bottom)
        for y in range(3, -1, -1):  # 3, 2, 1, 0 (top to bottom)
            print("│", end="")

            # Inner loop: iterate through columns (x-coordinates left to right)
            for x in range(0, 4):  # 0, 1, 2, 3 (left to right)
                loc = Location(x, y)

                # Collect all objects at this location
                objects = []

                if self.is_agent_at(loc):
                    agent_symbol = {"E": "A→", "W": "←A", "N": "A↑", "S": "A↓"}
                    objects.append(agent_symbol[self.agent_orientation.symbol()])

                if self.is_pit_at(loc):
                    objects.append("P")

                if self.is_wumpus_at(loc):
                    wumpus_symbol = "W" if self.wumpus_alive else "†W"
                    objects.append(wumpus_symbol)

                if self.is_gold_at(loc):
                    objects.append("G")

                # Format the cell with all objects
                if objects:
                    cell_content = ",".join(objects)
                    # Pad to center in the cell (adjust spacing as needed)
                    cell = f"{cell_content:^7}"
                else:
                    cell = "       "

                print(cell, end="│")

            # After finishing a row, print a newline and row separator
            print()  # End the current row

            if y > 0:  # Don't print separator after the last row
                print("├───────┼───────┼───────┼───────┤")

        print("└───────┴───────┴───────┴───────┘")

        # Rest of your status information remains the same...
        print(f"\nCumulative Reward: {self.cumulative_reward}")
        print(f"Step Reward: {self.last_reward}")
        print(f"Arrow: {'✓' if self.agent_has_arrow else '✗'}")
        print(f"Gold: {'✓' if self.agent_has_gold else '✗'}")
        print(f"Wumpus: {'Alive' if self.wumpus_alive else 'Dead'}")
        print(f"Game Over: {self.game_over}")

        print("\nPERCEPT:")
        if current_percept:
            print(f"Bump: {current_percept.bump}")
            print(f"Scream: {current_percept.scream}")
        else:
            print(f"Bump: False")
            print(f"Scream: False")
        print(f"Breeze: {self.is_breeze()}")
        print(f"Stench: {self.is_stench()}")
        print(f"Glitter: {self.is_glitter()}")

In [ ]:
env = Environment(0, False)
agent = NaiveAgent()
agent.run()

WUMPUS WORLD SIMULATION
LEGEND:
  A→ A← A↑ A↓  = Agent (facing direction)
  P             = Pit
  W             = Wumpus (alive)
  †W            = Wumpus (dead)
  G             = Gold
  Multiple objects in same cell shown as: A→,G

WUMPUS WORLD - Step 0
┌───────┬───────┬───────┬───────┐
│       │       │   G   │       │
├───────┼───────┼───────┼───────┤
│       │       │       │       │
├───────┼───────┼───────┼───────┤
│       │   W   │       │       │
├───────┼───────┼───────┼───────┤
│  A→   │       │       │       │
└───────┴───────┴───────┴───────┘

Cumulative Reward: 0
Step Reward: 0
Arrow: ✓
Gold: ✗
Wumpus: Alive
Game Over: False

PERCEPT:
Bump: False
Scream: False
Breeze: False
Stench: False
Glitter: False

WUMPUS WORLD - Step 1 - Action.FORWARD
┌───────┬───────┬───────┬───────┐
│       │       │   G   │       │
├───────┼───────┼───────┼───────┤
│       │       │       │       │
├───────┼───────┼───────┼───────┤
│       │   W   │       │       │
├───────┼───────┼───────┼───────